#### Demand Forecasting for Inventory Management Using ARIMA

To accurately forecast the future demand for products in a warehouse using the ARIMA model, enabling optimal inventory management and reducing costs associated with overstocking and stockouts.

In [ ]:
# spare cell to install packages
%pip install statsmodels

In [ ]:
# import necessary libraries

import pandas as pd
import numpy as np

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error

import matplotlib.pyplot as plt

In [ ]:
# sample data - synthetic time series dataset representing the monthly sales of a product over a few years

np.random.seed(42)

date_range = pd.date_range(start='2018-01-01', end='2023-12-31', freq='M')
sales = np.random.poisson(lam=200, size=len(date_range)) + np.arange(len(date_range)) * 5

# create a DataFrame
data = pd.DataFrame({'Date': date_range, 'Sales': sales})
data.set_index('Date', inplace=True)

data.head()

In [ ]:
# plot the time series

plt.figure(figsize=(10, 6))
plt.plot(data, label='Sales')
plt.title('Monthly Sales Data')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()

plt.show()

In [ ]:
# check for stationarity using the Augmented Dickey-Fuller (ADF) test

'''
The Augmented Dickey-Fuller test is a statistical test used to check whether a time series is stationary. 
Stationarity means that the statistical properties of the series, such as mean and variance, do not change over time. 
Most time series models, including ARIMA, assume that the series is stationary.
'''
result = adfuller(data['Sales'])

'''
ADF Statistic: A more negative value indicates stronger evidence against the presence of a unit root 
(i.e., stronger evidence that the series is stationary).

p-value: A lower p-value (< 0.05) indicates that you can reject the null hypothesis, suggesting 
that the series is stationary.
'''
print('ADF Statistic:', result[0])
print('p-value:', result[1])

# if not stationary, difference it
# differencing is a method where you subtract the previous observation from the current observation
data_diff = data.diff().dropna()


In [ ]:
# plot ACF and PACF

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
plot_acf(data_diff, ax=ax1, lags=20)
plot_pacf(data_diff, ax=ax2, lags=20)

plt.show()


In [ ]:
# arima model

model = ARIMA(data, order=(1, 1, 1))  # adjust p, d, q based on ACF and PACF
model_fit = model.fit()

print(model_fit.summary())

In [ ]:
# forecasts

forecast_steps = 12
forecast = model_fit.get_forecast(steps=forecast_steps)
forecast_index = pd.date_range(start=data.index[-1], periods=forecast_steps + 1, freq='M')[1:]
forecast_series = pd.Series(forecast.predicted_mean, index=forecast_index)
conf_int = forecast.conf_int()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(data, label='Observed')
plt.plot(forecast_series, label='Forecast', color='red')
plt.fill_between(forecast_index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='pink', alpha=0.3)
plt.title('Sales Forecast')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()

plt.show()

In [ ]:
# split into train and test sets

train_size = int(len(data) * 0.8)
train, test = data[:train_size], data[train_size:]

In [ ]:
# training arima model

model = ARIMA(train, order=(1, 1, 1))
model_fit = model.fit()


In [ ]:
# make predictions on test set

forecast = model_fit.forecast(steps=len(test))

In [ ]:
# evaluate the model

mae = mean_absolute_error(test, forecast)
mse = mean_squared_error(test, forecast)

print('MAE:', mae)
print('MSE:', mse)

Other random visualisations

In [ ]:
# differenced time series plot - helps to check if the differencing to achieve stationarity was effective

data_diff.plot(figsize=(10, 6))
plt.title('Differenced Time Series Plot')

plt.show()

In [ ]:
# residual plot - help to diagnose if the model residuals (errors) are random and normally distributed (if not, model can be improved)

residuals = model_fit.resid
residuals.plot(figsize=(10, 6))
plt.title('Residuals Plot')

plt.show()

In [ ]:
# forecast vs actual plot

plt.figure(figsize=(10, 6))
plt.plot(test, label='Actual')
plt.plot(forecast, label='Forecast', color='red')
plt.legend()
plt.title('Forecast vs. Actual')

plt.show()



Based on the above forecast vs actual plot, we can see that the horizontal line forecast is indicative of the ARIMA model not capturing any patterns from the data, such as trends or seasonality. As such, to improve our model, we can use SARIMA (Seasonal ARIMA) to account for seasonality.

In [ ]:
# import sarimax
from statsmodels.tsa.statespace.sarimax import SARIMAX

# seasonal parameters (P, D, Q, S)
seasonal_order = (1, 1, 1, 12)

model = SARIMAX(train, order=(1, 1, 1), seasonal_order=seasonal_order)
model_fit = model.fit(disp=False)


In [ ]:
# plot forecast vs actual graph

forecast = model_fit.forecast(steps=len(test))
plt.figure(figsize=(10, 6))
plt.plot(test, label='Actual')
plt.plot(forecast, label='Forecast', color='red')
plt.legend()
plt.title('SARIMA Forecast vs. Actual')

plt.show()


further improvements of the model

In [ ]:
# import necessary libraries

import itertools
import warnings
warnings.filterwarnings("ignore")
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
# parameter tuning - grid search to find the best parameters for ARIMA/SARIMA model

# define the p, d, and q parameters to take any value between 0 and 2
p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))

# seasonal parameters
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

best_aic = float("inf")
best_params = None

# for each pararmeter in the list
for param in pdq:
    for param_seasonal in seasonal_pdq:

        try:
            tmp_model = SARIMAX(train, order=param, seasonal_order=param_seasonal)
            tmp_model_fit = tmp_model.fit(disp=False)
            tmp_aic = tmp_model_fit.aic
            if tmp_aic < best_aic:
                best_aic = tmp_aic
                best_params = (param, param_seasonal)
                    
        except:
            continue

# print out best sarima/arima parameters
print(f'Best SARIMA parameters: {best_params}, AIC: {best_aic}')


In [ ]:
# evaluate performance of model

best_model = SARIMAX(train, order=best_params[0], seasonal_order=best_params[1])
best_model_fit = best_model.fit(disp=False)

mae = mean_absolute_error(test, forecast)
mse = mean_squared_error(test, forecast)
print('MAE:', mae)
print('MSE:', mse)

In [ ]:
# plot forecast graph

forecast = best_model_fit.forecast(steps=len(test))
plt.figure(figsize=(10, 6))
plt.plot(test, label='Actual')
plt.plot(forecast, label='Forecast', color='red')
plt.legend()
plt.title('Best SARIMA Forecast vs. Actual')
plt.show()